<a href="https://colab.research.google.com/github/ManjuRama/FinMath/blob/main/NV4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re

# Define the file path
file_path = "your_file.txt"  # Replace with your actual file path

# Initialize list to store rows of data and variables for tracking the current row
data_rows = []
current_row = {}
current_name = None
in_footer_section = False

# Regular expression pattern to capture numbered name-value pairs (e.g., '1. ISIN: ABCDEFG')
pattern = re.compile(r'^\d+\.\s*(\w+):\s*(.*)')

# Open and read the file
with open(file_path, 'r') as file:
    for line in file:
        line = line.strip()

        # Skip lines without a colon, assuming they are headers or footers
        if ':' not in line:
            # If we're already in the footer section, skip these lines
            if in_footer_section:
                continue
            # Otherwise, ignore this line as it's not a valid name-value pair
            continue

        # Match the line with the regular expression pattern to extract the name and value
        match = pattern.match(line)
        if match:
            # Extract the attribute name and value
            name, value = match.groups()
            name = name.strip()
            value = value.strip()

            # If we encounter an 'ISIN', it signals the start of a new row
            if name == "ISIN":
                # If there's an existing row with data, add it to data_rows
                if current_row:
                    data_rows.append(current_row)

                # Reset for a new ISIN section
                current_row = {"ISIN": value}
                current_name = "ISIN"
                in_footer_section = False  # Reset footer section flag
            else:
                # Add the name-value pair to the current row
                current_row[name] = value
                current_name = name
        else:
            # If the line does not match the pattern but contains text, it’s a continuation line
            if current_name and current_name in current_row:
                # Append the continuation line to the current attribute's value
                current_row[current_name] += ' ' + line.strip()

        # Check for a footer section after encountering ISIN and other attributes
        if line.lower().startswith("footer") or not match:
            in_footer_section = True

    # Add the last row if it has data
    if current_row:
        data_rows.append(current_row)

# Convert the list of rows into a DataFrame
df = pd.DataFrame(data_rows)

# Display the DataFrame
print(df)
